# M&E DFL Data Transpose

#### Importing required libraries

In [1]:
import pandas as pd, numpy as np
import xlsxwriter, openpyxl
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Required libraries imported!')

Required libraries imported!


#### Reading the excel / csv file

In [2]:
def csvORexcel(file_Name):
    global path
    path = "C:\\Python\\read\\"+file_Name
    try:
        if file_Name.split('.')[-1].startswith('c'):
            df = pd.read_csv(path)
            return df
        elif file_Name.split('.')[-1].startswith('x'):
            df = pd.read_excel(path)
            return df
    except FileNotFoundError:
        print("The file name {0} has not found".format(path))

#### Transposing cell data

In [3]:
def transposeDFL(df):
    remove_rows=[] # Initiating a empty list to store the index numbers of the rows which need to be removed after repeatation.

    for idx in df.index: # Iterating through index numbers.
        if df['RepresentativeNames'][idx] not in [np.nan]: # Checking if cell needs transpose.
            
            remove_rows.append(idx) # Storing the index numbers of the rows which need to be removed after repeatation.
            
            # Creating list of cell data against 'RepresentativeNames', 'RepresentativeRoles', 'RepresentativeMobileNumbers' columns.
            rName = df['RepresentativeNames'][idx].split(',') # Spliting cell data to convert into list.
            rRole = df['RepresentativeRoles'][idx].split(',') # Spliting cell data to convert into list.
            rMoNo = df['RepresentativeMobileNumbers'][idx].split(',') # Spliting cell data to convert into list.
    
            if len(rRole) > len(rName): # Checking if 'RepresentativeNames' have more data than 'RepresentativeRoles'.
                for i in range(len(rRole)-len(rName)): # Iterating through the difference of the no. of elements between 'RepresentativeNames' & 'RepresentativeRoles'
                    rRole.pop() # Remove extra entry from 'RepresentativeRoles'
            
            if len(rMoNo) > len(rName): # Checking if 'RepresentativeNames' have more data than 'RepresentativeMobileNumbers'.
                for i in range(len(rMoNo)-len(rName)): # Iterating through the difference of the no. of elements between 'RepresentativeNames' & 'RepresentativeMobileNumbers'
                    rMoNo.pop() # Remove extra entry from 'RepresentativeMobileNumbers'
            
            if len(rRole) < len(rName): # Checking if 'RepresentativeRoles' have more data than 'RepresentativeNames'.
                for i in range(len(rName)-len(rRole)): # Iterating through the difference of the no. of elements between 'RepresentativeNames' & 'RepresentativeRoles'
                    rRole.append(df['RepresentativeRoles'][idx].split(',')[-1].strip()) # Repeating the last element of the and adding it to 'RepresentativeRoles' to match with 'RepresentativeNames'
            
            if len(rMoNo) < len(rName): # Checking if 'RepresentativeMobileNumbers' have more data than 'RepresentativeNames'.
                for i in range(len(rName)-len(rMoNo)): # Iterating through the difference of the no. of elements between 'RepresentativeNames' & 'RepresentativeMobileNumbers'
                    rMoNo.append(df['RepresentativeMobileNumbers'][idx].split(',')[-1].strip()) # Repeating the last element of the and adding it to 'RepresentativeMobileNumbers' to match with 'RepresentativeNames'

            temp = pd.concat([df[idx:idx+1]] * len(rName), axis=0,ignore_index=True) # Repeating the row and temporarily storing it
            temp['RepresentativeNames'] = rName # Transposing the 'RepresentativeNames'
            temp['RepresentativeRoles'] = rRole # Transposing the 'RepresentativeRoles'
            temp['RepresentativeMobileNumbers'] = rMoNo # Transposing the 'RepresentativeMobileNumbers'
            df = pd.concat([df,temp],axis=0) # Adding transposed and repeated rows to main data.
                
    df.drop(index=remove_rows,inplace=True) # Remove initial rows which have transposed.
    return df # Return final transposed data.

def export_data(df): # Export data in excel format.
    with pd.ExcelWriter('C:\\Python\\export\\transposed_data_'+fn.split('.')[0]+'.xlsx', engine='xlsxwriter', engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        df.to_excel(writer, sheet_name='Transposed Data', index=False)
    print('Transposed Data has been exported to below location:\n','C:\\Python\\export\\transposed_data_'+fn.split('.')[0]+'.xlsx')

#### Processing the data

In [4]:
file_Name = input('Please provide file name = ')
data0 = csvORexcel(file_Name)
fn = file_Name
print('Transposing data of ',fn)
transposed_data = transposeDFL(data0)
print('Exporting transposed data of ',fn)
export_data(transposed_data)

Please provide file name =  DFL (19 Jan To 25 Jan) BR.xlsx


Transposing data of  DFL (19 Jan To 25 Jan) BR.xlsx
Exporting transposed data of  DFL (19 Jan To 25 Jan) BR.xlsx
Transposed Data has been exported to below location:
 C:\Python\export\transposed_data_DFL (19 Jan To 25 Jan) BR.xlsx
